In [38]:
import glob
import skimage.io as io
import numpy as np
#import commonfunctions as cf
import sklearn
from skimage.feature import hog
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
import cv2
import random
import os
from PIL import Image, ImageEnhance
from PIL import Image, ImageEnhance
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.decomposition import PCA

import pandas as pd



In [13]:

'''
if you want to load the data again to work with them
'''

male_data=np.load('male_data_small.npy',allow_pickle=True)
female_data = np.load('female_data_small.npy',allow_pickle=True)

#male_data_icdar=np.load('male_data_icdar.npy',allow_pickle=True)
#female_data_icdar=np.load('female_data_icdar.npy',allow_pickle=True)


In [14]:
#print(male_data_icdar.shape[0])
#print(female_data_icdar.shape[0])

#male_data=np.concatenate((male_data,male_data_icdar[:186]))
#female_data=np.concatenate((female_data,female_data_icdar))
#male_data=male_data_icdar
#female_data=female_data_icdar
#print(male_data.shape[0])
#print(female_data.shape[0])


278
286
278
286


In [15]:
from skimage import feature, exposure

def HistogramofOrientedGradients(image):
    resized_img = resize(image, (64, 128))
    fd, hog_image = feature.hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True)
    
    return (np.array(fd)).flatten()



In [16]:
# print(np.reshape(HistogramofOrientedGradients(male_data[0]),(1,-1)).shape)

# temp=HistogramofOrientedGradients(male_data[0])
# X=temp
# X=np.reshape(X,(-1,1))
# pca=PCA(n_components=10)
# X=pca.fit_transform(X)
# print(X)

In [17]:


N_ANGLE_BINS = 40
BIN_SIZE = 360 // N_ANGLE_BINS
LEG_LENGTH = 25

def get_contour_pixels(bw_image):
    contours, _= cv2.findContours(
        bw_image, cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_NONE
        ) 
    # contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]
    
    img2 = bw_image.copy()[:,:,np.newaxis]
    img2 = np.concatenate([img2, img2, img2], axis = 2)
    
    # if self.show_images:
    #     for cnt in contours : 
    #         cv2.drawContours(img2, [cnt], 0, (255, 0, 0), 1)  
            
    #     plt.imshow(img2, cmap='gray')
    return contours

def get_hinge_features(bw_image):
    # if self.is_binary:
    #     bw_image, _ = self.preprocess_binary_image(img_file, self.sharpness_factor, self.bordersize)
    # else:
    #     bw_image, _ = self.preprocess_image(img_file, self.sharpness_factor, self.bordersize)
    
    contours = get_contour_pixels(bw_image)
    
    hist = np.zeros((N_ANGLE_BINS, N_ANGLE_BINS))
        
    # print([len(cnt) for cnt in contours])
    for cnt in contours:
        n_pixels = len(cnt)
        if n_pixels <= LEG_LENGTH:
            continue
        
        points = np.array([point[0] for point in cnt])
        xs, ys = points[:, 0], points[:, 1]
        point_1s = np.array([cnt[(i + LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        point_2s = np.array([cnt[(i - LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        x1s, y1s = point_1s[:, 0], point_1s[:, 1]
        x2s, y2s = point_2s[:, 0], point_2s[:, 1]
        
        phi_1s = np.degrees(np.arctan2(y1s - ys, x1s - xs) + np.pi)
        phi_2s = np.degrees(np.arctan2(y2s - ys, x2s - xs) + np.pi)
        
        indices = np.where(phi_2s > phi_1s)[0]
        
        for i in indices:
            phi1 = int(phi_1s[i] // BIN_SIZE) % N_ANGLE_BINS
            phi2 = int(phi_2s[i] // BIN_SIZE) % N_ANGLE_BINS
            hist[phi1, phi2] += 1
            
    normalised_hist = hist / np.sum(hist)
    feature_vector = normalised_hist[np.triu_indices_from(normalised_hist, k = 1)]
    
    return feature_vector


#Initialization:
N_RHO_BINS = 7
N_ANGLE_BINS = 12
N_BINS = N_RHO_BINS * N_ANGLE_BINS
BIN_SIZE = 360 // N_ANGLE_BINS
R_INNER = 5.0
R_OUTER = 35.0
K_S = np.arange(3, 8)
def get_cold_features(bw_image, approx_poly_factor = 0.01 , sharpness_factor = 10 , bordersize = 3):

    #bw_image, _ = preprocess_image(img_file, sharpness_factor, bordersize)
    contours = get_contour_pixels(bw_image)
    
    rho_bins_edges = np.log10(np.linspace(R_INNER, R_OUTER, N_RHO_BINS))
    feature_vectors = np.zeros((len(K_S), N_BINS))
    
    for j, k in enumerate(K_S):
        hist = np.zeros((N_RHO_BINS, N_ANGLE_BINS))
        for cnt in contours:
            epsilon = approx_poly_factor * cv2.arcLength(cnt,True)
            cnt = cv2.approxPolyDP(cnt,epsilon,True)
            n_pixels = len(cnt)
            
            point_1s = np.array([point[0] for point in cnt])
            x1s, y1s = point_1s[:, 0], point_1s[:, 1]
            point_2s = np.array([cnt[(i + k) % n_pixels][0] for i in range(n_pixels)])
            x2s, y2s = point_2s[:, 0], point_2s[:, 1]
            
            thetas = np.degrees(np.arctan2(y2s - y1s, x2s - x1s) + np.pi)
            rhos = np.sqrt((y2s - y1s) ** 2 + (x2s - x1s) ** 2)
            if(rhos.all()):
                rhos_log_space = np.log10(rhos)
            else:
                rhos_log_space = rhos
            
            quantized_rhos = np.zeros(rhos.shape, dtype=int)
            for i in range(N_RHO_BINS):
                quantized_rhos += (rhos_log_space < rho_bins_edges[i])
                
            for i, r_bin in enumerate(quantized_rhos):
                theta_bin = int(thetas[i] // BIN_SIZE) % N_ANGLE_BINS
                hist[r_bin - 1, theta_bin] += 1
            
        normalised_hist = hist / hist.sum()
        feature_vectors[j] = normalised_hist.flatten()
        
    return feature_vectors.flatten()

    

In [18]:
print(get_hinge_features(male_data[0]).shape)
print(male_data[0].shape)

(66,)
(1420, 1000)


In [19]:
print(get_cold_features(male_data[0]).shape)
print(get_hinge_features(male_data[0]).shape)

(420,)
(66,)


In [20]:
a=np.array([1,2,3])
b=np.array([4,5,6])
c=np.array([4,5,6])
d=np.concatenate((a,b,c),axis=0)
print(d)

[1 2 3 4 5 6 4 5 6]


In [22]:


cold_male_feature=[]
cold_female_feature=[]
hinge_male_feature=[]
hinge_female_feature=[]
hog_male_feature=[]
hog_female_feature=[]
# pca=PCA(n_components=70)
# X=pca.fit_transform(X)
# print(X.shape, Y.shape)

#print(get_hinge_features(male_data[0]).shape)
for i,img in enumerate(male_data):
    
    cold_male_feature.append(get_cold_features(img))
    hinge_male_feature.append(get_hinge_features(img))
    hog_male_feature.append(HistogramofOrientedGradients(img))
    #male_feature.append(np.concatenate((cold,hinge),axis=0))
    # print(i)

for i,img in enumerate(female_data):
    
    cold_female_feature.append(get_cold_features(img))
    hinge_female_feature.append(get_hinge_features(img))
    hog_female_feature.append(HistogramofOrientedGradients(img))
    #female_feature.append(np.concatenate((cold,hinge),axis=0))

cold_male_feature=np.array(cold_male_feature)
hinge_male_feature=np.array(hinge_male_feature)
cold_female_feature=np.array(cold_female_feature)
hinge_female_feature=np.array(hinge_female_feature)
hog_male_feature=np.array(hog_male_feature)
hog_female_feature=np.array(hog_female_feature)

In [23]:

X_cold=np.concatenate((cold_male_feature,cold_female_feature),axis=0)
pca=PCA(n_components=70)
X_cold=pca.fit_transform(X_cold)

X_hinge=np.concatenate((hinge_male_feature,hinge_female_feature),axis=0)

X_hog=np.concatenate((hog_male_feature,hog_female_feature),axis=0)

X=np.concatenate((X_cold,X_hinge,X_hog),axis=1)


Y=np.concatenate((np.ones(cold_male_feature.shape[0]),
                np.zeros(cold_female_feature.shape[0])),
                axis=0)

print(X_cold[:,:70].shape)
print(X_hinge.shape)
print(X.shape)   


(564, 70)
(564, 66)
(564, 3916)


In [24]:
print(male_data.shape)    
print(female_data.shape)    

(278, 1420, 1000)
(286, 1420, 1000)


In [25]:
# pca=PCA(n_components=70)
# X=pca.fit_transform(X)
# print(X.shape, Y.shape)


In [31]:
X,Y=sklearn.utils.shuffle(X,Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42) # 70% training and 30% test


In [32]:
model_cold = svm.SVC(kernel='rbf', verbose=True, C=10,probability=True)

model_cold.fit(X_train[:,:70], y_train)
y_pred_cold=model_cold.predict(X_test[:,:70])
y_pred_prob_cold=model_cold.predict_proba(X_test[:,:70])
print(y_pred_prob_cold)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_cold))


[LibSVM]..*.*
optimization finished, #iter = 1409
obj = -819.346115, rho = -0.340672
nSV = 275, nBSV = 49
Total nSV = 275
..*.*
optimization finished, #iter = 1294
obj = -858.962093, rho = -0.579940
nSV = 267, nBSV = 68
Total nSV = 267
..*.*
optimization finished, #iter = 1320
obj = -865.154043, rho = -0.405973
nSV = 277, nBSV = 56
Total nSV = 277
..*.*
optimization finished, #iter = 1375
obj = -803.480006, rho = -0.075376
nSV = 264, nBSV = 49
Total nSV = 264
..*.*
optimization finished, #iter = 1358
obj = -744.424575, rho = -0.282495
nSV = 259, nBSV = 47
Total nSV = 259
..*.*
optimization finished, #iter = 1668
obj = -1053.763837, rho = 0.252481
nSV = 328, nBSV = 74
Total nSV = 328
[[0.19612912 0.80387088]
 [0.8105222  0.1894778 ]
 [0.26456562 0.73543438]
 [0.90320868 0.09679132]
 [0.24749326 0.75250674]
 [0.38769971 0.61230029]
 [0.95521753 0.04478247]
 [0.32733258 0.67266742]
 [0.19265229 0.80734771]
 [0.58741532 0.41258468]
 [0.35014367 0.64985633]
 [0.65554471 0.34445529]
 [0.6690

In [33]:
model_hinge = svm.SVC(kernel='rbf', verbose=True, C=10,probability=True)

model_hinge.fit(X_train[:,70:136], y_train)
y_pred_hinge=model_hinge.predict(X_test[:,70:136])
y_pred_prob_hinge=model_hinge.predict_proba(X_test[:,70:136])
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_hinge))


[LibSVM]..*.*
optimization finished, #iter = 1143
obj = -1751.422792, rho = -1.512264
nSV = 266, nBSV = 174
Total nSV = 266
..*.*
optimization finished, #iter = 1088
obj = -1658.352711, rho = -0.495340
nSV = 261, nBSV = 169
Total nSV = 261
.*.*
optimization finished, #iter = 973
obj = -1744.787531, rho = -1.503523
nSV = 264, nBSV = 174
Total nSV = 264
.*.*
optimization finished, #iter = 942
obj = -1638.446053, rho = -1.259617
nSV = 252, nBSV = 164
Total nSV = 252
.*.*
optimization finished, #iter = 944
obj = -1688.684632, rho = -1.598283
nSV = 261, nBSV = 173
Total nSV = 261
.*.*
optimization finished, #iter = 1041
obj = -2156.367749, rho = 1.242260
nSV = 318, nBSV = 218
Total nSV = 318
Accuracy: 0.7168141592920354


In [34]:
model_hog = svm.SVC(kernel='rbf', verbose=True, C=10,probability=True)

model_hog.fit(X_train[:,136:], y_train)
y_pred_hog=model_hog.predict(X_test[:,136:])
print(y_pred_hog)
y_pred_prob_hog=model_hog.predict_proba(X_test[:,136:])
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_hog))

[LibSVM].*.*
optimization finished, #iter = 903
obj = -442.236015, rho = -0.192810
nSV = 355, nBSV = 0
Total nSV = 355
.*.*
optimization finished, #iter = 900
obj = -440.980349, rho = -0.002614
nSV = 350, nBSV = 0
Total nSV = 350
.*.*
optimization finished, #iter = 948
obj = -460.437234, rho = 0.037690
nSV = 356, nBSV = 0
Total nSV = 356
.*.*
optimization finished, #iter = 906
obj = -443.745010, rho = -0.124580
nSV = 354, nBSV = 0
Total nSV = 354
.*.*
optimization finished, #iter = 904
obj = -437.880380, rho = -0.249475
nSV = 355, nBSV = 0
Total nSV = 355
.*.*
optimization finished, #iter = 1173
obj = -574.863601, rho = 0.162669
nSV = 440, nBSV = 0
Total nSV = 440
[0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1.
 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1

In [35]:
y_pred=np.argmax(y_pred_prob_hinge+y_pred_prob_cold+y_pred_prob_hog, axis=1)
#print(y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7522123893805309


In [36]:

#y_pred = clf.predict(X_test)

# print(y_pred)
# print(y_test)

In [37]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7522123893805309


In [37]:
accuracy=0
X=np.concatenate((male_feature,female_feature),axis=0)
Y=np.concatenate((np.ones(male_feature.shape[0]),
                    np.zeros(female_feature.shape[0])),
                    axis=0)
pca=PCA(n_components=200)
X=pca.fit_transform(X)
t=0
for i in range(0,100):


    X,Y=sklearn.utils.shuffle(X,Y)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=(i+1000)%5) # 70% training and 30% test
    clf = svm.SVC(kernel='rbf', verbose=True, C=10)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy+=metrics.accuracy_score(y_test, y_pred)

    
    pass
print(accuracy/100)

NameError: name 'male_feature' is not defined

In [34]:
print(male_feature.shape)

(232, 486)


In [ ]:
a=np.ones((3,3))
b=np.zeros((3,4))
print(a.shape,b.shape)
print(np.concatenate((a,b),axis=1))

(3, 3) (3, 4)
[[1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]]


In [ ]:
def cold_model(male_data,female_data):
    cold_male_feature=[]
    cold_female_feature=[]
    for i,img in enumerate(male_data):
        cold_male_feature.append(get_cold_features(img))

    for i,img in enumerate(female_data):        
        cold_female_feature.append(get_cold_features(img))
    cold_male_feature=np.array(cold_male_feature)
    cold_female_feature=np.array(cold_female_feature)
    X_cold=np.concatenate((cold_male_feature,cold_female_feature),axis=0)
    pca=PCA(n_components=70)
    X_cold=pca.fit_transform(X_cold)

    
        